# Flexibility Around Cost

## Imports and Definitions

In [ ]:
import os
import torch
import numpy as np
import cvxpy as cp
import pandas as pd
import IPython.display as display
%matplotlib inline

from SCMP import SCMP, FlexibleSCMP
import DataGeneration as data
import Presentation
import Visualization

torch.set_default_dtype(torch.float64)
torch.manual_seed(0)

PATH = "./Results/flexibility"
if not os.path.exists(PATH):
    os.makedirs(PATH)

## Data Generation

In [ ]:
x_dim = 2
n_samples = 640
X, Y = data.gen_custom_normal_data(x_dim, n_samples, torch.Tensor([0.5, 0]), torch.Tensor([0.1, 1]), torch.Tensor([-0.5, 0]), torch.Tensor([0.1, 1]))
positive = X[Y == 1]
negative = X[Y == -1]
pd.DataFrame(positive.numpy()).to_csv(f"{PATH}/positive.csv")
pd.DataFrame(negative.numpy()).to_csv(f"{PATH}/negative.csv")
X, Y, Xval, Yval, Xtest, Ytest = data.split_validation_test(X, Y)

## Training

In [ ]:
epochs = 16
batch_size = 16
reg = 0.001

v_0 = torch.tensor([0.5, 0.5])
v_best = torch.tensor([1, 0])

In [ ]:
# Non-strategic classification - Benchmark
print("---------- Training the benchmark model (non-strategically with v=v_0) ----------")
model_name = "benchmark"
non_strategic_model = SCMP(x_dim, batch_size, cost_fn="linear", cost_const_kwargs={"v": v_0}, strategic=False)
non_strategic_model.fit(X, Y, Xval, Yval, opt_class=torch.optim.Adam, opt_kwargs={"lr": 1e-1}, epochs=epochs, verbose="epochs", save_progression=True, path=PATH, model_name=model_name)

non_strategic_model = SCMP(x_dim, batch_size, cost_fn="linear", cost_const_kwargs={"v": v_0}, strategic=False)
non_strategic_model.load_model(PATH, model_name)

w = non_strategic_model.w.data
b = non_strategic_model.b.data
v = v_0
acc = non_strategic_model.evaluate(Xtest, Ytest, strategic_data=False)
results = [w[0], w[1], b.item(), v[0], v[1], acc]
pd.DataFrame(results, index=["w0", "w1", "b", "v0", "v1", "acc"]).to_csv(f"{PATH}/benchmark_results.csv")

# Strategic classification - Oracle
print("---------- Training the oracle model (strategically with v=v_best) ----------")
model_name = "oracle"
strategic_model_oracle = SCMP(x_dim, batch_size, cost_fn="linear", cost_const_kwargs={"v": v_best}, strategic=True)
strategic_model_oracle.fit(X, Y, Xval, Yval, opt_class=torch.optim.Adam, opt_kwargs={"lr": 1e-1}, epochs=epochs, verbose="epochs", save_progression=True, path=PATH, model_name=model_name)

strategic_model_oracle = SCMP(x_dim, batch_size, cost_fn="linear", cost_const_kwargs={"v": v_best}, strategic=True)
strategic_model_oracle.load_model(PATH, model_name)

w = strategic_model_oracle.w.data
b = strategic_model_oracle.b.data
v = v_best
acc = strategic_model_oracle.evaluate(Xtest, Ytest, strategic_data=True)
results = [w[0], w[1], b.item(), v[0], v[1], acc]
pd.DataFrame(results, index=["w0", "w1", "b", "v0", "v1", "acc"]).to_csv(f"{PATH}/oracle_results.csv")

# Strategic classification - Naive
print("---------- Training the naive model (strategically with v=v_0) ----------")
model_name = "naive"
strategic_model_naive = SCMP(x_dim, batch_size, cost_fn="linear", cost_const_kwargs={"v": v_0}, strategic=True)
strategic_model_naive.fit(X, Y, Xval, Yval, opt_class=torch.optim.Adam, opt_kwargs={"lr": 1e-1}, epochs=epochs, verbose="epochs", save_progression=True, path=PATH, model_name=model_name)

strategic_model_naive = SCMP(x_dim, batch_size, cost_fn="linear", cost_const_kwargs={"v": v_0}, strategic=True)
strategic_model_naive.load_model(PATH, model_name)

w = strategic_model_naive.w.data
b = strategic_model_naive.b.data
v = v_0
acc = strategic_model_naive.evaluate(Xtest, Ytest, strategic_data=True)
results = [w[0], w[1], b.item(), v[0], v[1], acc]
pd.DataFrame(results, index=["w0", "w1", "b", "v0", "v1", "acc"]).to_csv(f"{PATH}/naive_results.csv")

# Strategic classification - SERM
print("---------- Training the SERM model (strategically and flexible with v_init=v_0) ----------")
model_name = "SERM"
strategic_model_SERM = FlexibleSCMP(x_dim, batch_size, v_init=v_0, reg_factor=reg, strategic=True)
strategic_model_SERM.fit(X, Y, Xval, Yval, opt_class=torch.optim.Adam, opt_kwargs={"lr": 1e-1}, epochs=epochs, verbose="epochs", save_progression=True, path=PATH, model_name=model_name)

strategic_model_SERM = FlexibleSCMP(x_dim, batch_size, v_init=v_0, reg_factor=reg, strategic=True)
strategic_model_SERM.load_model(PATH, model_name)

w = strategic_model_SERM.w.data
b = strategic_model_SERM.b.data
v = strategic_model_SERM.v.data
v /= torch.norm(v)
acc = strategic_model_SERM.evaluate(Xtest, Ytest, strategic_data=True)
results = [w[0], w[1], b.item(), v[0], v[1], acc]
pd.DataFrame(results, index=["w0", "w1", "b", "v0", "v1", "acc"]).to_csv(f"{PATH}/SERM_results.csv")

## Show Results

In [ ]:
Presentation.show_flexibility_results()